In [1]:
import tensorflow as tf

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 使用tf.match_filename_once函数获取文件列表
files = tf.train.match_filenames_once('.\data.tfrecords-*')

# 通过tf.train.string_input_producer函数创建输入队列，输入队列中的文件列表为tf.train.match_filenames_once函数获取的文件列表
# 这里将shuffle参数设置为False来避免随机打乱读文件的顺序。但一般在解决真实问题时，会将shuffle参数设置为True

filename_queue = tf.train.string_input_producer(files, shuffle=False)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'i': tf.FixedLenFeature([], tf.int64),
                                       'j': tf.FixedLenFeature([], tf.int64),
                                   })

example, label = features['i'], features['j']
# 一个batch中样例的个数
batch_size = 3
capacity = 1000 + 3 * batch_size

In [5]:
# 使用tf.train.batch函数来组合样例。[example， label]参数给出了需要组合的元素，一般example和label分别代表训练样本和这个样本对应的正确标签。
# 当队列长度等于容量时，Tensorflow将暂停入队操作，而只是等待出队。当元素个数小于容量时，Tensorflow将自动启动入队操作
example_batch, label_batch = tf.train.batch([example, label], batch_size=batch_size, capacity=capacity)

In [6]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print('-->',cur_example_batch, cur_label_batch)
    coord.request_stop()
    coord.join(threads)

--> [0 1 0] [1 1 1]
--> [1 0 1] [1 1 1]
